# Infrastructure Setup

In [ ]:
import os
from chi import context

# We select which chameleon site we are using and choose the project
context.use_site("CHI@UC")
context.choose_project()
username = os.environ.get('USER').replace("_","-")

## Creating a 1-day lease

In [ ]:
from chi import lease
from datetime import timedelta

In [ ]:
node_type="compute_cascadelake_r"
instance_count = 2

In [ ]:
mpi_lease = lease.Lease(f"{username}-mpi-lease", duration=timedelta(days=1))

mpi_lease.add_node_reservation(amount=instance_count, node_type=node_type)
mpi_lease.add_fip_reservation(1)
mpi_lease.submit(idempotent=True)

## Creating server instances

In [ ]:
from chi import server

In [ ]:
mpi_master=server.Server(
        f"{username}-mpi-master",
        reservation_id=mpi_lease.node_reservations[0]["id"],
        image_name="Ubuntu22.04-HPC-MPI-Spack",
        network_name="sharednet1"
)
mpi_master.submit(idempotent=True)

mpi_workers = []
for i in range(instance_count - 1):
    mpi_worker=server.Server(
            f"{username}-mpi-worker-{i+1}",
            reservation_id=mpi_lease.node_reservations[0]["id"],
            image_name="Ubuntu22.04-HPC-MPI-Spack",
            network_name="sharednet1"
    )
    mpi_worker.submit(idempotent=True)
    mpi_workers.append(mpi_worker)

In [ ]:
fip = mpi_lease.get_reserved_floating_ips()[0]
mpi_master.associate_floating_ip(fip)